![Servers](../images/servers.png)
I wanted to deploy a cloud instance in order to perform High Frequency Trading wiht low latency toward a couple of cryptocurrency exchanges.
First of all I needed a list of exchanges, so I leveraged an open source library which allows an unified API toward those exchanges.
It definitively have a list of endpoits.

In [ ]:
import pandas as pd
import ccxt
from importlib import import_module

ccxt_modules = [mod for mod in dir(ccxt) if mod[0].islower() and mod not in 
                ['base','decimal_to_precision','error_hierarchy',
                 'errors','exchanges','static_dependencies']]
rows = []
for mod in ccxt_modules:
    full_module_name = "ccxt." + mod

    imp_module = import_module(full_module_name)
    describe = getattr(imp_module, mod)().describe()
    api_elem = describe['urls']['api']
    if isinstance(api_elem, str):
        api_url = api_elem
    else:
        assert isinstance(api_elem, dict)
        if 'private' in api_elem.keys():
            api_url = api_elem['private']
        elif 'v3Private' in api_elem.keys():
            api_url = api_elem['v3Private']
        elif 'trade' in api_elem.keys():
            api_url = api_elem['trade']
        elif 'api' in api_elem.keys():
            api_url = api_elem['api']
        elif 'public' in api_elem.keys():
            api_url = api_elem['public']
        elif 'rest' in api_elem.keys():
            api_url = api_elem['rest']
        elif 'publicV2' in api_elem.keys():
            api_url = api_elem['publicV2']
        elif 'current' in api_elem.keys():
            api_url = api_elem['current']
        elif len(api_elem) == 1:
            api_url = api_elem[api_elem.keys()[0]]
        else:
            print("Unexpected json structure")
    if 'hostname' in api_url:
        api_url = api_url.format(hostname=describe['hostname'])

    rows.append([describe['id'], api_url])

exchanges_df = pd.DataFrame(rows, columns=['name','api_url']).set_index('name')
exchanges_df

In [2]:
from urllib.parse import urlparse
import socket

def extract_domain(url):
    domain = urlparse(url).hostname
    return domain

def get_ip(domain):
    try:
        return socket.gethostbyname(domain)
    except Exception as e:
        # print(domain)
        # print(e)
        pass

exchanges_df['api_url_domain'] = exchanges_df.api_url.dropna().apply(extract_domain)
exchanges_df['api_ip'] = exchanges_df['api_url_domain'].dropna().apply(get_ip)
exchanges_df

,api_url,api_url_domain,api_ip
name,,,
aax,https://api.aax.com,api.aax.com,52.220.66.226
acx,https://acx.io/api,acx.io,3.106.77.227
aofex,https://openapi.aofex.com/openApi,openapi.aofex.com,104.20.161.135
bequant,https://api.bequant.io,api.bequant.io,172.67.74.135
bibox,https://api.bibox365.com,api.bibox365.com,104.26.13.161
...,...,...,...
xbtce,https://cryptottlivewebapi.xbtce.net:8443/api,cryptottlivewebapi.xbtce.net,195.154.26.204
xena,https://api.xena.exchange,api.xena.exchange,104.22.73.120
yobit,https://yobit.net/tapi,yobit.net,104.16.241.98


Then I used a tool for grouping togheter ip addresses in Autonomous Systems. 
In order to do that I downloaded MRT/RIB BGP archives from Routeviews (or similar sources).
[link](https://github.com/hadiasghari/pyasn)

In [3]:
import pyasn

asndb = pyasn.pyasn('../data/ipasn.20201106.dat')

def _get_asn(ip):
    return asndb.lookup(ip)[1] or ip


exchanges_df['api_asn'] = exchanges_df['api_ip'].dropna().apply(_get_asn)
exchanges_df

,api_url,api_url_domain,api_ip,api_asn
name,,,,
aax,https://api.aax.com,api.aax.com,52.220.66.226,52.220.0.0/15
acx,https://acx.io/api,acx.io,3.106.77.227,3.104.0.0/14
aofex,https://openapi.aofex.com/openApi,openapi.aofex.com,104.20.161.135,104.20.160.0/20
bequant,https://api.bequant.io,api.bequant.io,172.67.74.135,172.67.64.0/20
bibox,https://api.bibox365.com,api.bibox365.com,104.26.13.161,104.26.0.0/20
...,...,...,...,...
xbtce,https://cryptottlivewebapi.xbtce.net:8443/api,cryptottlivewebapi.xbtce.net,195.154.26.204,195.154.0.0/16
xena,https://api.xena.exchange,api.xena.exchange,104.22.73.120,104.22.64.0/20
yobit,https://yobit.net/tapi,yobit.net,104.16.241.98,104.16.240.0/20


In [4]:
import requests

url = 'https://ip-ranges.amazonaws.com/ip-ranges.json'
req = requests.get(url)

import pandas as pd

aws_ip_ranges_df = pd.DataFrame(req.json()['prefixes'],  
                                columns=['ip_prefix', 'region', 'service', 'network_border_group'])
aws_ip_ranges_df.set_index('ip_prefix', inplace=True)
aws_ip_ranges_df

,region,service,network_border_group
ip_prefix,,,
3.5.140.0/22,ap-northeast-2,AMAZON,ap-northeast-2
35.180.0.0/16,eu-west-3,AMAZON,eu-west-3
52.93.178.234/32,us-west-1,AMAZON,us-west-1
52.94.76.0/22,us-west-2,AMAZON,us-west-2
52.95.36.0/22,ap-southeast-2,AMAZON,ap-southeast-2
...,...,...,...
44.242.161.8/31,us-west-2,KINESIS_VIDEO_STREAMS,us-west-2
44.242.184.128/25,us-west-2,AMAZON,us-west-2
52.43.76.88/29,us-west-2,CODEBUILD,us-west-2


In [5]:
exchanges_df = exchanges_df.merge(aws_ip_ranges_df, how='left', left_on='api_asn', right_index=True)
exchanges_df

,api_url,api_url_domain,api_ip,api_asn,region,service,network_border_group
name,,,,,,,
aax,https://api.aax.com,api.aax.com,52.220.66.226,52.220.0.0/15,ap-southeast-1,AMAZON,ap-southeast-1
aax,https://api.aax.com,api.aax.com,52.220.66.226,52.220.0.0/15,ap-southeast-1,EC2,ap-southeast-1
acx,https://acx.io/api,acx.io,3.106.77.227,3.104.0.0/14,ap-southeast-2,AMAZON,ap-southeast-2
acx,https://acx.io/api,acx.io,3.106.77.227,3.104.0.0/14,ap-southeast-2,EC2,ap-southeast-2
aofex,https://openapi.aofex.com/openApi,openapi.aofex.com,104.20.161.135,104.20.160.0/20,NaN,NaN,NaN
...,...,...,...,...,...,...,...
xena,https://api.xena.exchange,api.xena.exchange,104.22.73.120,104.22.64.0/20,NaN,NaN,NaN
yobit,https://yobit.net/tapi,yobit.net,104.16.241.98,104.16.240.0/20,NaN,NaN,NaN
zaif,https://api.zaif.jp,api.zaif.jp,13.112.252.96,13.112.0.0/14,ap-northeast-1,AMAZON,ap-northeast-1


In [67]:
exchanges_df.groupby(['region']).count()[['api_url']]\
    .rename(mapper=dict(api_url='count'), axis=1)\
    .sort_values('count', ascending=False).index.values

array(['ap-northeast-1', 'eu-west-1', 'ap-northeast-2', 'ap-southeast-1',
       'us-east-1', 'ap-southeast-2', 'eu-central-1'], dtype=object)